In [1]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run, ir_datasets
import pyterrier as pt
import os

In [2]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-lab-sose-2024/ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset('irds:' + dataset)

pt_index_path = './terrier-index'


if not os.path.exists(pt_index_path + "/data.properties"):
  # create the index, using the IterDictIndexer indexer 
  indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

  # we give the dataset get_corpus_iter() directly to the indexer
  # while specifying the fields to index and the metadata to record
  index_ref = indexer.index(pt_dataset.get_corpus_iter(), 
                            meta=('docno',))

else:
  # if you already have the index, use it.
  index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

# Running a Grid Search Algorithm to find the best configuration for BM25
We will use the best configuration afterwards.

In [5]:
def run_bm25_grid_search_run(index, output_dir, queries):
    """
        defaults: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/BM25.html
        k_1 = 1.2d, k_3 = 8d, b = 0.75d
        We do not tune parameter k_3, as this parameter only impacts queries with reduntant terms.
    """
    for b in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for k_1 in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0]:
            system = f'bm25-b={b}-k_1={k_1}'
            configuration = {"bm25.b" : b, "bm25.k_1": k_1}
            run_output_dir = output_dir + '/' + system
            !rm -Rf {run_output_dir}
            !mkdir -p {run_output_dir}
            print(f'Run {system}')
            BM25 = pt.BatchRetrieve(index, wmodel="BM25", controls=configuration, verbose=True)
            run = BM25(queries)
            persist_and_normalize_run(run, system, run_output_dir)

In [6]:
dataset_tira = ir_datasets.load(dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(dataset), format='trecxml')
run_bm25_grid_search_run(index, 'grid-search/training', queries)

Run bm25-b=0.1-k_1=0.5


BR(BM25): 100%|██████████| 68/68 [00:04<00:00, 15.61q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.1-k_1=0.5".
Done. run file is stored under "grid-search/training/bm25-b=0.1-k_1=0.5/run.txt".
Run bm25-b=0.1-k_1=0.6


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 42.72q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=0.1-k_1=0.6".
Done. run file is stored under "grid-search/training/bm25-b=0.1-k_1=0.6/run.txt".
Run bm25-b=0.1-k_1=0.7


...

The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=1.0-k_1=2.8".
Done. run file is stored under "grid-search/training/bm25-b=1.0-k_1=2.8/run.txt".
Run bm25-b=1.0-k_1=2.9


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 43.60q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=1.0-k_1=2.9".
Done. run file is stored under "grid-search/training/bm25-b=1.0-k_1=2.9/run.txt".
Run bm25-b=1.0-k_1=3.0


BR(BM25): 100%|██████████| 68/68 [00:01<00:00, 45.83q/s]


The run file is normalized outside the TIRA sandbox, I will store it at "grid-search/training/bm25-b=1.0-k_1=3.0".
Done. run file is stored under "grid-search/training/bm25-b=1.0-k_1=3.0/run.txt".


# Evaluate all Configurations from Grid Search

In [9]:
!pip3 install trectools

from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()

def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-sose-2024', dataset, truth_dataset=True) + '/qrels.txt')

training_qrels = load_qrels('ir-acl-anthology-20240504-training')


def evaluate_run(run_dir, qrels):
    run = TrecRun(run_dir + '/run.txt')
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank(),
        'P@10': trec_eval.get_precision(depth=10)
    }


df = []
for r in glob('grid-search/training/bm25*'):
    df += [evaluate_run(r, training_qrels)]
df = pd.DataFrame(df)
df_sorted = df.sort_values('nDCG@10', ascending=False)
print(df_sorted)
best_configuration = df_sorted['run'].values[0]
best_b = best_configuration.split("-")[1].split("=")[1]
best_k_1 = best_configuration.split("-")[2].split("=")[1]

/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()
/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current 

KeyboardInterrupt: 

In [6]:
best_b = 0.1
best_k_1 = 1.9

bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls= {"bm25.b" : best_b, "bm25.k_1": best_k_1})
sdm = pt.rewrite.SDM()

# Query expansion/rewriting with LLM´s

In [10]:
# Load the expansions
dataset_tira = ir_datasets.load(dataset)
queries = pt.io.read_topics(ir_datasets.topics_file(dataset), format='trecxml')
# llm expansions with gpt
gpt_cot = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-cot', dataset, prefix='llm_expansion_')
gpt_sq_fs = tira.pt.transform_queries('workshop-on-open-web-search/tu-dresden-03/qe-gpt3.5-sq-fs', dataset, prefix='llm_expansion_')
gpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with llama
llama_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-cot', dataset, prefix='llm_expansion_')
llama_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-fs', dataset, prefix='llm_expansion_')
llama_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-llama-sq-zs', dataset, prefix='llm_expansion_')

# llm expansions with flan-ul2
flan_cot = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-cot', dataset, prefix='llm_expansion_')
flan_sq_fs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-fs', dataset, prefix='llm_expansion_')
flan_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-flan-ul2-sq-zs', dataset, prefix='llm_expansion_')

In [11]:
topics = pt_dataset.get_topics(variant='title')

tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], topic['llm_expansion_query']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)


pipeline_gpt_cot = (gpt_cot >> pt_expand_query) >> bm25
pipeline_gpt_sq_fs = (gpt_sq_fs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25

pipeline_llama_cot = (llama_cot >> pt_expand_query) >> bm25
pipeline_llama_sq_fs = (llama_sq_fs >> pt_expand_query) >> bm25
pipeline_llama_sq_zs = (llama_sq_zs >> pt_expand_query) >> bm25

pipeline_flan_cot = (flan_cot >> pt_expand_query) >> bm25
pipeline_flan_sq_fs = (flan_sq_fs >> pt_expand_query) >> bm25
pipeline_flan_sq_zs = (flan_sq_zs >> pt_expand_query) >> bm25



In [43]:
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
    
topics = pt_dataset.get_topics(variant='title')

SEED=42

!pip3 install scikit-learn

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)

ltr_feats1 = bm25 >> pt.text.get_text(pt_dataset, ["text", "doc_id"]) >> (
    pt.BatchRetrieve(index, wmodel="Js_KLs")
    ** # KL Expansion
    pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "KL"})
    ** # ChatGPT Similiar Queries Few Shot
    pipeline_gpt_sq_fs
    ** # ChatGPT Similiar Queries Zero shot
    pipeline_gpt_sq_zs
    ** # Llama-2 Chain of Thought
    pipeline_llama_cot
    ** # Llama-2 Similiar Queries Zero shot
    pipeline_llama_sq_zs
    ** # Flan-UL2 Similiar Queries Few Shot
    pipeline_flan_sq_fs
)

# for reference, lets record the feature names here too
fnames=["Js_KLs", "KL", "gpt_sq_fs", "gpt_sq_zs", "llama_cot", "llama_sq_zs", "flan_sq_fs"]

In [44]:
from sklearn.ensemble import RandomForestRegressor
!pip3 install joblib
import joblib

model_path = "./model9.joblib"

if os.path.exists(model_path):
    # Load the model
    model = joblib.load(model_path)
    print("Model loaded from file.")
    rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(model)
else:
    
    model = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=4)
    rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(model)
    rf_pipe.fit(train_topics, pt_dataset.get_qrels())
    joblib.dump(model, model_path)
    print("Model trained and saved to file.")

print(model.feature_importances_)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   34.7s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:  1.2min finished


Model trained and saved to file.
[0.19679015 0.19760398 0.20677358 0.20676063 0.19207166]


In [45]:
def save_results(system, name):
    run = system(pt_dataset.get_topics('text'))
    persist_and_normalize_run(run, system_name=name, default_output='../runs')
    os.rename('../runs/run.txt', "../runs/"+name+".txt")

save_results(rf_pipe, "15_samples_even_even_less_features")
# run = rf_pipe(pt_dataset.get_topics('text'))
# persist_and_normalize_run(run, system_name="RandomForestRegressor", default_output='../runs')

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.9s finished


The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
